🌡️ Temperature Classification using DHT22
Classes: Cold – Warm – Hot
1️⃣ Problem Definition (Very Important)
Sensor

DHT22 temperature sensor

Output: Temperature in °C (continuous value)

Goal

Convert a continuous temperature into discrete classes:Cold

Warm

Hot

👉 This is a CLASSIFICATION problem, not regression.

2️⃣ Define the Classes (Domain Knowledge)

You must define ranges first (this is important for teaching).

Example class definition
Class	Temperature Range (°C)
Cold	< 20
Warm	20 – 30
Hot	> 30

📌 You can adjust these ranges based on location.

3️⃣ Dataset Creation (Simple & Realistic)
Example dataset format
Temperature (°C)	Label
16.5	Cold
22.0	Warm
28.5	Warm
32.0	Hot
35.5	Hot

Python label encoding

In [26]:
def label_temperature_norm(temp_norm):
    temp = temp_norm * 50.0  # convert back to °C
    if temp < 20:
        return 0   # Cold
    elif temp <= 30:
        return 1   # Warm
    else:
        return 2   # Hot


Class mapping:

In [27]:
class_names = ["Cold", "Warm", "Hot"]

Create Training Data (Synthetic or Real)

In [28]:
import numpy as np
import sklearn

# Generate synthetic temperature data
temps = np.random.uniform(10, 40, 600)
temps_norm = temps / 50.0

labels = np.array([label_temperature_norm(t) for t in temps_norm])

X = temps_norm.reshape(-1, 1)
y = labels


In [5]:
# Normalize (DHT22 max range ≈ 50°C)
X = X / 50.0

⚠️ Golden Rule:

Same normalization must be applied on STM32 / ESP32.
5️⃣ Split Data (Train / Test)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

6️⃣ Build Embedded-Friendly Model

Minimal neural network (perfect for MCUs):

In [13]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(1,)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])


model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 16)             │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 195 (780.00 B)

 Trainable params: 195 (780.00 B)

 Non-trainable params: 0 (0.00 B)

7️⃣ Train the Model

In [21]:
history = model.fit(
    X_train, y_train,
    epochs=150,
    batch_size=16,
    validation_split=0.1,
    verbose=1
)

Epoch 1/150
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9222 - loss: 0.5159 - val_accuracy: 0.9000 - val_loss: 0.5148
Epoch 2/150
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9194 - loss: 0.5063 - val_accuracy: 0.9250 - val_loss: 0.5031
Epoch 3/150
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9250 - loss: 0.5028 - val_accuracy: 0.9250 - val_loss: 0.5138
Epoch 4/150
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9167 - loss: 0.5021 - val_accuracy: 0.9250 - val_loss: 0.5108
Epoch 5/150
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9444 - loss: 0.4915 - val_accuracy: 0.9250 - val_loss: 0.5074
Epoch 6/150
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9389 - loss: 0.4859 - val_accuracy: 0.9750 - val_loss: 0.4904
Epoch 7/150
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9361 - loss: 0.4813 - val_accuracy: 0.9250 - val_loss: 0.4947
Epoch 8/150
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9417 - loss: 0.4795 - val_accuracy: 0.9250 - v

8️⃣ Evaluate the Model

In [22]:
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.2134 
Test Accuracy: 1.0


9️⃣ Test with Known Temperatures (Demo Cell)

In [29]:
test_temps = np.array([15, 25, 35]).reshape(-1, 1)
test_temps_norm = test_temps / 50.0

pred = model.predict(test_temps_norm)
pred_classes = np.argmax(pred, axis=1)

for t, c in zip(test_temps.flatten(), pred_classes):
    print(f"{t} °C → {class_names[c]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
15 °C → Hot
25 °C → Hot
35 °C → Hot


In [20]:
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{np.int64(0): np.int64(175),
 np.int64(1): np.int64(163),
 np.int64(2): np.int64(162)}

🔟 Confidence Interpretation (Very Important)

In [24]:
confidence = np.max(pred, axis=1)

for t, conf in zip(test_temps.flatten(), confidence):
    print(f"Confidence: {conf:.2f}")


Confidence: 1.00
Confidence: 1.00
Confidence: 1.00


In [25]:
pred = model.predict(test_temps_norm)
pred_class = np.argmax(pred, axis=1)
confidence = np.max(pred, axis=1)

for t, c, conf in zip(test_temps.flatten(), pred_class, confidence):
    if conf < 0.7:
        print(f"{t} °C → Uncertain")
    else:
        print(f"{t} °C → {class_names[c]} ({conf:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
15 °C → Hot (1.00)
25 °C → Hot (1.00)
35 °C → Hot (1.00)


1️⃣1️⃣ Convert to TensorFlow Lite (For Embedded)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

with open("dht22_temp_classifier.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model size:", len(tflite_model), "bytes")
